# Clustering Zooniverse Marks to count Iguanas
Count all the iguanas in the images by clustering the marks from the zooniverse volunteers.
This does not compare the results to the gold standard and requires only a single file, the flattened zooniverse data export.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
This notebook is used to cluster the marks from the zooniverse volunteers to count the iguanas in the images saved in results/<phase_tag>/flat_dataset_filtered_Iguanas 3rd launch.csv

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

## Load the data

In [2]:
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold



# Phase Selection
# phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = None # None or a number, filter records which have less than these user interactions.
use_gold_standard_subset = None # Use no filtering


## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")
# input_path = Path("results/")
# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_04_12_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)
reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Location for plots
# output_plot_path = output_path.joinpath("plots")
# output_plot_path.mkdir(parents=True, exist_ok=True)
output_plot_path = None # do not plot

## Look into the config
This Config points to all files necessary for the analysis + the result files

In [3]:
from zooniverse.config import get_config

config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-3rdphase_renamed.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/3-T2-GS-results-5th-0s.csv'),
 'image_source': None,
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 3rd launch/yes_no_dataset_Iguanas 3rd launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 3rd launch/flat_dataset_Iguanas 3rd launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 3rd launch/flat_panoptes_points_Iguanas 3rd launch.csv'),
 'panoptes_question': PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 3rd launch/panoptes_question

In [4]:
config["flat_panoptes_points"]

PosixPath('/Users/christian/data/zooniverse/2024_04_12_analysis/Iguanas 3rd launch/flat_panoptes_points_Iguanas 3rd launch.csv')

## Look at the data


In [5]:

df_flat_panoptes_points = pd.read_csv(config["flat_panoptes_points"])
df_flat_panoptes_points["phase"] = phase_tag

df_flat_panoptes_points

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
0,430180580,096835c7b506ed9511c344f420d01f74,002400ef36f94c5e2a6ccc49859923d8,22040,T2,2022-08-01 20:06:32 UTC,78963883,FPA03_54.jpg,449,399,Iguanas 3rd launch
1,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,511,594,Iguanas 3rd launch
2,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,349,711,Iguanas 3rd launch
3,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,255,756,Iguanas 3rd launch
4,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,9,245,Iguanas 3rd launch
...,...,...,...,...,...,...,...,...,...,...,...
106322,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,267,31,Iguanas 3rd launch
106323,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,221,52,Iguanas 3rd launch
106324,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,198,31,Iguanas 3rd launch
106325,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,582,152,Iguanas 3rd launch


### Filter User if necessary and Marks


In [6]:
# Use the panoptes Extraction instead of the custom extraction
df_merged_dataset = df_flat_panoptes_points

### Are there anonymous users in the data?
There should be some because anonymous users are kept.

In [7]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
91417,447704338,18905a576ccd1be7a6a14fa96a7bb04d,NaN,22040,T2,2022-10-31 02:25:07 UTC,78921874,GEB01-1_128.jpg,753,791,Iguanas 3rd launch
91418,447704338,18905a576ccd1be7a6a14fa96a7bb04d,NaN,22040,T2,2022-10-31 02:25:07 UTC,78921874,GEB01-1_128.jpg,767,849,Iguanas 3rd launch
91419,447704338,18905a576ccd1be7a6a14fa96a7bb04d,NaN,22040,T2,2022-10-31 02:25:07 UTC,78921874,GEB01-1_128.jpg,673,907,Iguanas 3rd launch
91420,447704338,18905a576ccd1be7a6a14fa96a7bb04d,NaN,22040,T2,2022-10-31 02:25:07 UTC,78921874,GEB01-1_128.jpg,351,863,Iguanas 3rd launch
91421,447704338,18905a576ccd1be7a6a14fa96a7bb04d,NaN,22040,T2,2022-10-31 02:25:07 UTC,78921874,GEB01-1_128.jpg,335,816,Iguanas 3rd launch
...,...,...,...,...,...,...,...,...,...,...,...
106322,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,267,31,Iguanas 3rd launch
106323,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,221,52,Iguanas 3rd launch
106324,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,198,31,Iguanas 3rd launch
106325,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,582,152,Iguanas 3rd launch


In [8]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

7663

In [9]:
## After filtering there
df_merged_dataset

,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y,phase
0,430180580,096835c7b506ed9511c344f420d01f74,002400ef36f94c5e2a6ccc49859923d8,22040,T2,2022-08-01 20:06:32 UTC,78963883,FPA03_54.jpg,449,399,Iguanas 3rd launch
1,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,511,594,Iguanas 3rd launch
2,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,349,711,Iguanas 3rd launch
3,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,255,756,Iguanas 3rd launch
4,501567924,9c249e3b4c1adc212dc055fabe626cfd,00346ebf6ae91002059d21fa7090e46b,22040,T2,2023-07-16 21:09:04 UTC,78925071,GWB01-1_218.jpg,9,245,Iguanas 3rd launch
...,...,...,...,...,...,...,...,...,...,...,...
106322,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,267,31,Iguanas 3rd launch
106323,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,221,52,Iguanas 3rd launch
106324,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,198,31,Iguanas 3rd launch
106325,494172012,fab341928dc32883a4e5ffcbf12977af,NaN,22040,T2,2023-06-06 12:35:40 UTC,78965185,ESCH02-2_92.jpg,582,152,Iguanas 3rd launch


In [10]:
# how many marks per user
df_merged_dataset[["user_id", "user_name", "x"]].groupby("user_name").count()

,user_id,x
user_name,,
00131d01aa119d2bb5f7e583bbc89842,0,11
0014724e6af5acc8cca2f508154fdba9,3,3
0014fa55f6b3afea1e75f78281ef02f3,0,82
001a7e5e989807349d0dc5e90eaebf83,4,4
0033c3f6ee86dee38a157162a8683621,17,17
...,...,...
ffa31db924ebf1353aaac21d4c6a3ff1,0,10
ffe917227e143f8531efccb9e6cb1e98,0,28
ffef396b5baaae379e21084f67aaa136,4,4


## Clustering

### Basic Statics like mean, median, mode

In [11]:
from zooniverse.analysis import get_mark_overview

basic_stats = []

for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"], 
                                                         subject_id=df_image_name.iloc[0]["subject_id"])

    ### basic statistics like mean, median, mode
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    
df_comparison = df_basic_stats


In [12]:
df_basic_stats

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,GEB01-1_101.jpg,78921850,1.0,1.00,1,1,[1],1.0,1,1,[1]
1,GEB01-1_102.jpg,78921851,2.0,2.00,2,2,[2],2.0,1,2,[2]
2,GEB01-1_109.jpg,78921856,3.0,3.00,2,4,"[2, 4]",3.0,4,12,"[2, 2, 4, 4]"
3,GEB01-1_112.jpg,78921860,6.0,6.00,6,6,[6],6.0,1,6,[6]
4,GEB01-1_113.jpg,78921861,1.0,1.00,1,1,[1],1.0,1,1,[1]
...,...,...,...,...,...,...,...,...,...,...,...
7658,ESCH02-2_79.jpg,78965181,1.0,1.00,1,1,[1],1.0,6,6,"[1, 1, 1, 1, 1, 1]"
7659,ESCH02-2_80.jpg,78965182,1.0,1.00,1,1,[1],1.0,1,1,[1]
7660,ESCH02-2_81.jpg,78965183,2.0,1.95,2,2,[2],2.0,20,39,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
7661,ESCH02-2_91.jpg,78965184,1.0,1.00,1,1,[1],1.0,1,1,[1]


In [13]:
# There might be records with too few annotations if they were not removed before
df_comparison[(df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
5461,PCIE07-3_153.jpg,78959385,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
4623,PCIC01-3_167.jpg,78939248,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
4365,PCI01-4_67.jpg,78938510,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
7534,ESCH02-1_300.jpg,78964988,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
6727,FSCA05-2_208.jpg,78963529,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...
2555,MEBBA03-2_114.jpg,78932214,1.0,1.0,1,1,[1],1.0,1,1,[1]
2556,MEBBA03-2_129.jpg,78932215,1.0,1.0,1,1,[1],1.0,1,1,[1]
2559,MEBBA03-2_162.jpg,78932224,2.0,2.0,2,2,[2],2.0,1,2,[2]
2560,MEBBA03-2_163.jpg,78932225,4.0,4.0,4,4,[4],4.0,1,4,[4]


In [14]:
# Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


### DBSCAN clustering and take the variant with the best silouette score for each image


In [ ]:
## sort by silouette score sorting + dbscan_count
from zooniverse.analysis import compare_dbscan

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    # check for at least 5 marks
    if df_image_name.shape[0] >= 5:
        dbscan_localization = compare_dbscan(
            params=params,
            df_flat=df_image_name,
            output_plot_path=output_plot_path,
            plot=show_plots
        )
    
        db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
        
        # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
        # Sillouette Scoring needs a minimum of 2 clusters
        # if there are points in decent radius they will belong to a cluster
        if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
            # If two or more cluster seem to exists take ones with the best Silouette score
        else:  
            # take the best result by silouette score if there are more clusters then 1
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)


In [ ]:
df_scan_best_results

Here it can be seen why the silouette score is difficult because it is often undefined.

In [ ]:
## save the combinations of parameters, which maximized the silouette score.
df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])


In [ ]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil"}, inplace=True)


df_comparison = df_comparison.merge(df_scan_best_results.drop(["image_name"], axis=1), on='subject_id', how='left')

In [ ]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

### HDBSCAN clustering for each image


In [ ]:
from zooniverse.analysis import hdbscan

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)
    
    image_name=df_image_name.iloc[0]["image_name"]
    subject_id=df_image_name.iloc[0]["subject_id"]
    
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=image_name, 
                                                         subject_id=subject_id)
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = hdbscan(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     subject_id=subject_id,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [ ]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

In [ ]:
## Statistics for all images
df_comparison = df_comparison.merge(df_hdbscan.drop(["image_name"], axis=1), on=['subject_id'], how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

In [ ]:
df_panoptes_question = pd.read_csv(config["panoptes_question"])
df_panoptes_question

In [ ]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

## Join the Expert Dataset for comparison
This dataset contains total_counts found by experts. 

In [ ]:
df_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")
# check what is in there
df_expert_count[df_expert_count.image_name == "SFM01-1-2_264.jpg"]

In [ ]:
## join the gold standard data to the basic stats
df_comparison_expert = df_expert_count[["site_name", "subject_group", "image_name", "subject_id", "count_total"]].merge(df_comparison.drop(["image_name"], axis=1), on='subject_id', how='left')

columns_to_fill = ['median_count', 'mean_count', 'mode_min_count', 'mode_max_count',
       'mode_count', 'mode_count_avg', 'users', 'sum_annotations_count',
        'dbscan_count_sil', 'HDBSCAN_count']
df_comparison_expert[columns_to_fill] = df_comparison_expert[columns_to_fill].fillna(0)

df_comparison_expert.sort_values(by="median_count", ascending=False)

In [ ]:
df_comparison_expert.to_csv(config["comparison_dataset_expert"])

In [ ]:
df_comparison_yes_no = df_comparison_expert.merge(df_panoptes_question, on="subject_id", how="left")
df_comparison_yes_no

In [ ]:
df_comparison_yes_no.to_csv(config["comparison_dataset_yes_no"])
df_comparison_yes_no

# A look into the results

## The sum of the clustering
What is the sum of the methods

In [ ]:
df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5]

In [ ]:
# Take only the subjects which have more or equal 5 yes votes.
df_method_sums = df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5][["count_total", "median_count", "mean_count", "mode_min_count", "mode_max_count", "mode_count_avg", "mode_min_count", "mode_max_count", "dbscan_count_sil", "HDBSCAN_count"]].sum()
df_method_sums.to_csv(config["method_sums"])
df_method_sums

## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [ ]:
print(f"{config['method_sums'].name}")
df_method_sums = pd.read_csv(config["method_sums"])
df_method_sums

### Comparison per Image Level

In [ ]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])